In [13]:
import pandas as pd
import numpy as np


# Load the uploaded CSV file to examine its structure
drive_path = '/content/drive/MyDrive/GTNP_Borehole_Dataset/Cleaned'
file_path = drive_path + '/consolidated_data_master.csv'
data = pd.read_csv(file_path)

reshaped = (
    data.set_index(["Date", "Depth"])[["Avg Ground Temp"]]
    .apply(lambda row: row.iloc[0], axis=1)  # Unpack the array and get the value
    .unstack(level=1)  # Unstack by Depth to make Depth the columns
)

# Convert the temperatures from Kelvin to Celsius
reshaped = reshaped - 273.15

# Rename columns with depth values for clarity
reshaped.columns = [f"{col}m" for col in reshaped.columns]

# Add the "Date" index back as a column for saving or displaying
reshaped.reset_index(inplace=True)

# Extract unique monthly air temperatures and convert to Celsius
monthly_air_temp = (
    data.groupby("Date")["Air Temp"].first() - 273.15
).reset_index()

# Rename the column for clarity
monthly_air_temp.rename(columns={"Air Temp": "Air"}, inplace=True)

# Merge the air temperature data into the reshaped DataFrame
reshaped = pd.merge(monthly_air_temp, reshaped, on="Date")

# Rename the Date column to Time
reshaped.rename(columns={"Date": "Time"}, inplace=True)

# Convert the Time column to a datetime format
reshaped['Time'] = pd.to_datetime(reshaped['Time'])

# Set Time as the index
reshaped.set_index('Time', inplace=True)

# Resample the data to a daily frequency and interpolate
daily_reshaped = reshaped.resample('D').interpolate(method="linear").fillna(method="bfill").fillna(method='ffill')

# Reset the index to include Time as a column
daily_reshaped.reset_index(inplace=True)

# Save the daily table to a CSV file
output_path = drive_path + '/Observations_daily_interpolated.csv'  # Update with your desired save path
daily_reshaped.to_csv(output_path, index=False)

# Print confirmation
print(f"Daily table saved to: {output_path}")



<ipython-input-13-355ed2301072>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_reshaped = reshaped.resample('D').interpolate(method="linear").fillna(method="bfill").fillna(method='ffill')


Daily table saved to: /content/drive/MyDrive/GTNP_Borehole_Dataset/Cleaned/Observations_daily_interpolated.csv


In [ ]:
"""
# Define constants
rho_soil_particles = 2650  # kg/m³
c_soil = 800  # J/kg·K
rho_water = 1000  # kg/m³
c_water = 4186  # J/kg·K
rho_bulk = 1500  # kg/m³

# Calculate porosity
n = 1 - (rho_bulk / rho_soil_particles)

# Extract soil moisture as a numpy array for vectorized computation
theta = data['Soil Moisture'].values  # Volumetric water content

# Vectorized thermal conductivity calculation
k_dry = 0.25  # W/m·K
k_sat = 1.5  # W/m·K
thermal_conductivity = k_dry + (k_sat - k_dry) * theta

# Vectorized volumetric heat capacity calculation
heat_capacity = (rho_bulk * c_soil) * (1 - theta) + (rho_water * c_water) * theta

# Soil density is constant as rho_bulk
density = np.full_like(theta, rho_bulk)

# Combine results into a dataframe
thermal_properties = pd.DataFrame({
    'Depth': data['Depth'],
    'Thermal Conductivity': thermal_conductivity,
    'Heat Capacity': heat_capacity,
    'Density': density
})

# Handle duplicate depths: aggregate by mean if required
aggregated_properties = thermal_properties.groupby('Depth').mean().reset_index()

# Save the aggregated results to a file (if GIPL needs unique depths)
aggregated_soil_properties_file = drive_path + '/aggregated_soil_properties_input.txt'
aggregated_properties.to_csv(aggregated_soil_properties_file, index=False, sep='\t')

# Save the full results (if GIPL supports multiple depth entries)
full_file_path =drive_path + '/per_time_instance_soil_proporties_input.txt'
thermal_properties.to_csv(full_file_path, index=False, sep='\t')
"""

In [ ]:
input_path = drive_path + '/GIPLOriginalInputFiles'

# Output files
updated_bound_file = input_path + "/updated_bound.txt"
updated_grid_file = input_path + "/updated_grid.txt"
updated_initial_file = input_path + "/updated_initial.txt"
updated_mineral_file = input_path + "/updated_mineral.txt"
updated_organic_file = input_path + "/updated_organic.txt"
updated_rsnow_file = input_path + "/updated_rsnow.txt"
updated_snow_file = input_path + "/updated_snow.txt"
updated_sites_file = input_path + "/updated_sites.txt"

##Bound.txt

In [ ]:
data = pd.read_csv(file_path)

# Converting the Date column to datetime for proper resampling
data['Date'] = pd.to_datetime(data['Date'])

# Resampling data to monthly intervals and averaging Air Temp
monthly_avg = data.resample('ME', on='Date')['Air Temp'].mean().reset_index()
monthly_avg['Air Temp'] = monthly_avg['Air Temp'] - 273.15

# Creating the bound.txt structure
bound_txt = []
bound_txt.append(str(len(monthly_avg)))  # Number of observations (month intervals)
bound_txt += [f"{i+1}\t{temp:.3f}" for i, temp in enumerate(monthly_avg['Air Temp'])]

# Combine into a single string for the bound.txt content
bound_txt_content = "\n".join(bound_txt)

# Save to file for validation or further usage
output_path = updated_bound_file
with open(output_path, "w") as file:
    file.write(bound_txt_content)

##Grid.txt

In [ ]:
# Define above ground and below ground depths as per the new ranges
above_ground = np.arange(-1.5, 0, 0.05)  # Above ground depths in 0.05m steps
below_ground = np.concatenate([
    np.arange(0, 0.01, 0.005),  # Very fine intervals near surface
    np.arange(0.01, 0.1, 0.01),  # Slightly coarser intervals
    np.arange(0.1, 1, 0.02),  # Coarser intervals at intermediate depths
    np.arange(1, 20, 0.1)  # Coarser intervals for deeper layers
])

# Combine above and below ground points
grid_points = np.concatenate((above_ground, below_ground))
grid_points = np.round(grid_points, 5)  # Match the precision in the example

# Output points example from the reference file
output_points = [39, 48, 51, 55, 58, 62, 66, 70, 74, 81, 89, 97]

# Prepare the grid.txt content
grid_txt = []
grid_txt.append(str(len(grid_points)))  # Total number of grid points
grid_txt += [f"{point}" for point in grid_points]  # Grid points
grid_txt.append(str(len(output_points)))  # Number of output points
grid_txt += [str(point) for point in output_points]  # Indices of output points

# Combine into a single string for the grid.txt content
grid_txt_content = "\n".join(grid_txt)

# Save to file for validation or further usage
output_path = updated_grid_file
with open(output_path, "w") as file:
    file.write(grid_txt_content)

##Initial.txt

In [ ]:
# Group by depth and month, selecting the first instance for each depth in a month
first_instance_data = data.sort_values(by=['Date']).groupby(['Depth', pd.Grouper(key='Date', freq='ME')]).first().reset_index()

# For the initial condition, we take the first occurrence of each depth
initial_condition_data = first_instance_data.groupby('Depth').first().reset_index()
initial_condition_data['Avg Ground Temp'] = initial_condition_data['Avg Ground Temp'] - 273.15

# Prepare initial.txt content
initial_txt = []
initial_txt.append("1   {}".format(len(initial_condition_data)))  # First row with number of points
initial_txt.append(" DEPTH  TEMP")  # Header
initial_txt += [f"{row.Depth:.3f}\t{row['Avg Ground Temp']:.3f}" for _, row in initial_condition_data.iterrows()]

# Combine into a single string for the initial.txt content
initial_txt_content = "\n".join(initial_txt)

# Save to file for validation or further usage
output_path = updated_initial_file
with open(output_path, "w") as file:
    file.write(initial_txt_content)

##Mineral.txt

In [ ]:
data = pd.read_csv(file_path)

# Define depth ranges for each layer (in meters)
layer_ranges = [
    (0, 0.21),    # Layer 1
    (0.21, 0.36), # Layer 2
    (0.36, 0.96), # Layer 3
]

# Initialize list to store layer properties
layers = []

# Iterate over each depth range to calculate properties
for depth_range in layer_ranges:
    layer_data = data[(data['Depth'] >= depth_range[0]) & (data['Depth'] < depth_range[1])]
    if not layer_data.empty:
        # Calculate average Volumetric Water Content (WVC)
        wvc = layer_data['Soil Moisture'].mean()
        # Assign typical values for 'a' and 'b' coefficients
        a_coefficient = 0.05
        b_coefficient = -0.2
        # Assign standard volumetric heat capacities
        thawed_heat_capacity = 2.75e6  # J/(m³·K)
        frozen_heat_capacity = 1.75e6  # J/(m³·K)
        # Assign standard thermal conductivities
        thawed_conductivity = 1.5  # W/(m·K)
        frozen_conductivity = 2.0  # W/(m·K)
        # Calculate layer thickness
        thickness = depth_range[1] - depth_range[0]
        # Append the calculated properties to the layers list
        layers.append((
            wvc,
            a_coefficient,
            b_coefficient,
            thawed_heat_capacity,
            frozen_heat_capacity,
            thawed_conductivity,
            frozen_conductivity,
            thickness
        ))

# Prepare the content for mineral.txt
mineral_txt = []
mineral_txt.append(" 1")  # First row, ignored by the model
mineral_txt.append(f"1  {len(layers)}")  # Second row: '1' ignored, second element is the number of layers
# Add each layer's properties
for layer in layers:
    line = "\t".join(f"{value:.5f}" if isinstance(value, float) else str(value) for value in layer)
    mineral_txt.append(line)

# Combine all lines into a single string
mineral_txt_content = "\n".join(mineral_txt)

# Save to mineral.txt file
output_path = updated_mineral_file
with open(output_path, "w") as file:
    file.write(mineral_txt_content)

##Organic.txt (Identical as given)

In [ ]:
organic_txt_content = \
""" 1
 1  0
"""

# Save to file for validation or further usage
output_path = updated_organic_file
with open(output_path, "w") as file:
    file.write(organic_txt_content)

##Snow.txt (We do not have our own data, using the average value of their results)

In [ ]:
data = pd.read_csv(file_path)

# Ensure the 'Date' column is in datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Extract unique months from the data
data['YearMonth'] = data['Date'].dt.to_period('M')  # Extract year and month as a period
unique_months = data['YearMonth'].unique()
unique_months_count = len(unique_months)  # Count unique months

# Load the reference snow.txt to calculate the average snow depth
reference_snow_path = input_path + '/snow.txt'  # Replace with the reference snow.txt file path
with open(reference_snow_path, 'r') as file:
    reference_snow_data = file.readlines()

# Extract snow depth values from the reference file (ignoring the first line)
reference_snow_depths = [
    float(line.split()[1]) for line in reference_snow_data[1:]
]

# Calculate the average snow depth from the reference data
average_snow_depth = sum(reference_snow_depths) / len(reference_snow_depths)

# Generate snow data with the correct number of observations and index numbers
snow_txt = [str(unique_months_count)]  # Number of observations
snow_txt += [f"{i+1}\t{average_snow_depth:.3f}" for i in range(unique_months_count)]

# Combine into a single string for snow.txt content
snow_txt_content = "\n".join(snow_txt)

# Save to file for validation or further usage
output_path = updated_snow_file  # Replace with your desired output path
with open(output_path, "w") as file:
    file.write(snow_txt_content)


##rsnow.txt (similar to above)

In [ ]:
data = pd.read_csv(file_path)

# Calculate the number of unique months in the CSV file
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')  # Ensure 'Date' is datetime
data = data.dropna(subset=['Date'])  # Drop invalid dates
data['YearMonth'] = data['Date'].dt.to_period('M')  # Extract year and month
unique_months_count = data['YearMonth'].nunique()  # Count unique months

# Set a constant thermal conductivity value from the reference rsnow.txt
constant_snow_conductivity = 0.3  # W/(mK), from reference, their example

# Generate rsnow.txt content
rsnow_txt = [str(unique_months_count)]  # First line: number of observations
rsnow_txt += [f"{i+1}\t{constant_snow_conductivity:.3f}" for i in range(unique_months_count)]

# Combine into a single string for rsnow.txt content
rsnow_txt_content = "\n".join(rsnow_txt)

# Save to file for validation or further usage
output_path = updated_rsnow_file
with open(output_path, "w") as file:
    file.write(rsnow_txt_content)


##sites.txt (Nothing changed, just making it output identically just for reference)

In [ ]:
# Generate sites.txt content using the example provided with temp_grd = 0.0
sites_txt_content = """1
1   1   1   1   1   0.0
"""

# Save to file for validation or further usage
output_path = updated_sites_file
with open(output_path, "w") as file:
    file.write(sites_txt_content)